# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key, weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = os.path.join('output_data', 'cities.csv')
df = pd.read_csv(file_path)
# Transform date data to a more readable format.
df['Date'] = pd.to_datetime(df['Date'], unit='s')
# Reformatted the date and removed the time information
df['Date'] = df['Date'].dt.strftime('%m/%d/%Y')
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bathsheba,13.22,-59.52,82.40,78,40,13.87,BB,07/14/2020
1,marienburg,54.04,19.03,54.00,100,0,3.36,PL,07/14/2020
2,butaritari,3.07,172.79,81.03,78,97,12.19,KI,07/14/2020
3,carnarvon,-24.87,113.63,73.40,64,75,12.75,AU,07/14/2020
4,port elizabeth,-33.92,25.57,48.20,87,40,33.33,ZA,07/14/2020
...,...,...,...,...,...,...,...,...,...
563,bosanska krupa,44.88,16.15,56.10,75,77,2.13,BA,07/14/2020
564,monte gordo,37.18,-7.45,73.40,88,0,5.82,PT,07/14/2020
565,pierre,44.37,-100.35,75.00,49,1,16.11,US,07/14/2020
566,laredo,27.51,-99.51,89.60,43,1,16.11,US,07/14/2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = df[['Lat', 'Lng']]
humidity = df['Humidity']

In [5]:
# Populate gmap figure
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
df2 = df.loc[df['Max Temp'] > 95]
# Found no null values

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = df2
hotel_df['Hotel Name'] = ''
hotel_df = hotel_df.reset_index()
hotel_df

C:\Users\szerp\Anacoda3\envs\PythonData2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,20,beyneu,45.32,55.20,98.89,17,0,15.28,KZ,07/14/2020,
1,52,saint george,37.10,-113.58,98.60,12,1,9.17,US,07/14/2020,
2,204,yilan,24.76,121.75,98.60,56,40,11.41,TW,07/14/2020,
3,221,mecca,21.43,39.83,95.09,15,67,4.29,SA,07/14/2020,
4,270,longhua,23.61,114.19,101.35,39,62,2.10,CN,07/14/2020,
5,275,dalbandin,28.89,64.41,97.86,14,0,3.04,PK,07/14/2020,
6,432,longyan,25.11,117.02,100.33,36,71,1.25,CN,07/14/2020,
7,550,phu ly,20.54,105.91,97.43,46,19,5.91,VN,07/14/2020,
8,559,zhanaozen,43.30,52.80,97.45,16,0,14.83,KZ,07/14/2020,


In [8]:
# Find closest hotel

# Iterate through dataframe
for index, row in hotel_df.iterrows():

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
    "location": f"{row['Lat']},{row['Lng']}",
    "radius": "5000",
    "keyword": "hotel",
    "key": g_key,
    }

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City'].title()}.")
    response = requests.get(base_url, params=params).json()
  
    results = response['results']

    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index,'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")        
        
    print("------------")

Retrieving Results for Index 0: Beyneu.
Closest hotel is HOTEL "AKNUR".
------------
Retrieving Results for Index 1: Saint George.
Closest hotel is Best Western Plus Abbey Inn.
------------
Retrieving Results for Index 2: Yilan.
Closest hotel is Silks Place Yilan.
------------
Retrieving Results for Index 3: Mecca.
Closest hotel is Hilton Makkah Convention Hotel.
------------
Retrieving Results for Index 4: Longhua.
Closest hotel is Taixianxuan Apartment Hotel.
------------
Retrieving Results for Index 5: Dalbandin.
Closest hotel is Al-Dawood Hotel & Restaurant.
------------
Retrieving Results for Index 6: Longyan.
Closest hotel is Longyan Ocean Business Hotel.
------------
Retrieving Results for Index 7: Phu Ly.
Closest hotel is Vinpearl Condotel Phủ Lý.
------------
Retrieving Results for Index 8: Zhanaozen.
Closest hotel is Hotel BasTau.
------------


In [9]:
hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,20,beyneu,45.32,55.20,98.89,17,0,15.28,KZ,07/14/2020,"HOTEL ""AKNUR"""
1,52,saint george,37.10,-113.58,98.60,12,1,9.17,US,07/14/2020,Best Western Plus Abbey Inn
2,204,yilan,24.76,121.75,98.60,56,40,11.41,TW,07/14/2020,Silks Place Yilan
3,221,mecca,21.43,39.83,95.09,15,67,4.29,SA,07/14/2020,Hilton Makkah Convention Hotel
4,270,longhua,23.61,114.19,101.35,39,62,2.10,CN,07/14/2020,Taixianxuan Apartment Hotel
5,275,dalbandin,28.89,64.41,97.86,14,0,3.04,PK,07/14/2020,Al-Dawood Hotel & Restaurant
6,432,longyan,25.11,117.02,100.33,36,71,1.25,CN,07/14/2020,Longyan Ocean Business Hotel
7,550,phu ly,20.54,105.91,97.43,46,19,5.91,VN,07/14/2020,Vinpearl Condotel Phủ Lý
8,559,zhanaozen,43.30,52.80,97.45,16,0,14.83,KZ,07/14/2020,Hotel BasTau


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]
hotel_locations

,Lat,Lng
0,45.32,55.20
1,37.10,-113.58
2,24.76,121.75
3,21.43,39.83
4,23.61,114.19
5,28.89,64.41
6,25.11,117.02
7,20.54,105.91
8,43.30,52.80


In [11]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

markers = gmaps.marker_layer(hotel_locations)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))